In [0]:
# Let's start by installing our libraries
# %pip install --quiet databricks-genai==1.0.8 mlflow==2.14.2
# %pip install --quiet databricks-sdk==0.29.0
# dbutils.library.restartPython()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow-skinny 2.17.0 requires databricks-sdk<1,>=0.20.0, but you have databricks-sdk 0.18.0 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-genai 1.0.8 requires databricks-sdk==0.18.0, but you have databricks-sdk 0.29.0 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Load the table back into a DataFrame
final_assigned_hcps_df = spark.table("final_assigned_hcps")
# Limit the DataFrame to 15 rows
limited_df = final_assigned_hcps_df.limit(15)

In [0]:
system_prompt = """
### Instruction
You are a fun, knowledgeable sales assistant. Based on the following Healthcare Provider (HCP) information, create a personalized, engaging briefing pack to help a sales agent prepare for their meeting. Responses should be brief, humorous, and human-like. Use icons wherever possible, and provide **Prescription Patterns** in table format. Present the data as a story, making it engaging for the agent.

**Important**:
- Use emojis/icons for each section header.
- Format **Prescription Patterns** into a table with Drug and Prescription count.
- Always include a **friendly greeting**, **inspirational quote**, and a **motivational conclusion**.

### Conversation Flow:

1. **Greeting**:
   Start with a cheerful greeting like, "Hello {Agent_Name}, ready to impress {HCP_Name}?"

2. **Inspire**: 
   Include a quote like, "Sales are contingent upon the attitude of the salesman, not the attitude of the prospect." – W. Clement Stone

### Briefing Pack Sections:

1. **👤 HCP Profile**
    - Name: {HCP_Name}
    - Specialty: {Specialty}
    - Distance from you: {Distance} km
    - Prefers: {Preferred_Meeting_Type}, Communication: {Communication_Pref}
    
2. **💊 Prescribing Behavior**
    - {HCP_Name} has been prescribing the following recently:
    
    | Drug        | Prescriptions   |
    |-------------|-----------------|
    | Drug A      | {Drug_A_Prescriptions} |
    | Drug B      | {Drug_B_Prescriptions} |
    | Drug C      | {Drug_C_Prescriptions} |
    | Other Drugs | {Other_Drugs_Prescriptions} |

3. **📅 Interaction History**
    - Last Meeting Outcome: {Meeting_Outcome}
    - Products Presented: {Product_Presented}
    - Recent Notes: {Notes}
    
4. **🌟 Product Preferences**
    - Preferred Drug: {hcp_Preferred_Drug}
    - Therapeutic Focus: {Therapeutic_Focus}
    
5. **🤝 Recommendations for Upcoming Interaction**
    - Focus on: {Agent_product_Focus}
    - Keep it light and engaging with {HCP_Name}, prefer {Preferred_Meeting_Type}, and communicate via {Communication_Pref}.
    
### Conclusion:
End by wishing the agent all the best, such as: "You’ve got this! All the best!"
"""


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import array, struct, lit, col, concat_ws

# Verify that all required columns are present
required_columns = [
    "HCP_Name",
    "Specialty",
    "Preferred_Drug",
    "Communication_Pref",
    "Preferred_Meeting_Type",
    "Meeting_Outcome",
    "Product_Presented",
    "Agent_Name",
    "Interaction_Type",
    "Therapeutic_Focus",
    "Product_focus",
    "Distance_km",
    "Drug_A_Prescriptions",
    "Drug_B_Prescriptions",
    "Drug_C_Prescriptions",
    "Other_Drugs_Prescriptions",
    "Notes",
    "Avg_Daily_Patients"
]

#Handle Nulls and Data Types
# Replace nulls in string columns with 'Unknown' and numerical columns with 0
for column in required_columns:
    if limited_df.schema[column].dataType.simpleString() == "string":
        limited_df = limited_df.fillna({column: "Unknown"})
    else:
        limited_df = limited_df.fillna({column: 0})


#Create the 'prompt' Column by Concatenating Relevant Columns
# This will serve as the user input for the chat completion
briefing_df = limited_df.withColumn(
    "prompt",
    concat_ws(
        "\n",
        lit("HCP Details:"),
        concat_ws(": ", lit("Name"), col("HCP_Name")),
        concat_ws(": ", lit("Specialty"), col("Specialty")),
        concat_ws(": ", lit("Preferred Drug"), col("Preferred_Drug")),
        concat_ws(": ", lit("Communication Preference"), col("Communication_Pref")),
        concat_ws(": ", lit("Preferred Meeting Type"), col("Preferred_Meeting_Type")),
        concat_ws(": ", lit("Meeting Outcome"), col("Meeting_Outcome")),
        concat_ws(": ", lit("Product Presented"), col("Product_Presented")),
        concat_ws(": ", lit("Interaction Type"), col("Interaction_Type")),
        concat_ws(": ", lit("Therapeutic Focus"), col("Therapeutic_Focus")),
        concat_ws(": ", lit("Agent Name"), col("Agent_Name")),
        concat_ws(": ", lit("Agent Product Focus"), col("Product_focus")),
        concat_ws(": ", lit("Distance"), concat_ws(" ", col("Distance_km"), lit("km"))),
        concat_ws(": ", lit("Drug A Prescriptions"), col("Drug_A_Prescriptions")),
        concat_ws(": ", lit("Drug B Prescriptions"), col("Drug_B_Prescriptions")),
        concat_ws(": ", lit("Drug C Prescriptions"), col("Drug_C_Prescriptions")),
        concat_ws(": ", lit("Other Drugs Prescriptions"), col("Other_Drugs_Prescriptions")),
        concat_ws(": ", lit("Notes"), col("Notes")),
        concat_ws(": ", lit("Average Daily Patients"), col("Avg_Daily_Patients"))
    )
)
final_df = briefing_df.select("Agent_ID","Agent_Name","HCP_ID","HCP_Name","prompt")

display(final_df)

Agent_ID,Agent_Name,HCP_ID,HCP_Name,prompt
AGT336,Agent_336,HCP001,Dr. Smith,HCP Details: Name: Dr. Smith Specialty: Oncology Preferred Drug: Drug_A Communication Preference: In-Person Preferred Meeting Type: In-Person Meeting Outcome: Negative Product Presented: Drug_B Interaction Type: Phone Therapeutic Focus: Oncology Agent Name: Agent_336 Agent Product Focus: Drug_A Distance: 135.0 km Drug A Prescriptions: 10 Drug B Prescriptions: 27 Drug C Prescriptions: 80 Other Drugs Prescriptions: 26 Notes: Provided samples of Drug_B. Average Daily Patients: 50
AGT843,Agent_843,HCP002,Dr. Johnson,HCP Details: Name: Dr. Johnson Specialty: Oncology Preferred Drug: Drug_C Communication Preference: Email Preferred Meeting Type: In-Person Meeting Outcome: Neutral Product Presented: Drug_B Interaction Type: Virtual Therapeutic Focus: Oncology Agent Name: Agent_843 Agent Product Focus: Drug_C Distance: 162.0 km Drug A Prescriptions: 14 Drug B Prescriptions: 15 Drug C Prescriptions: 31 Other Drugs Prescriptions: 23 Notes: HCP expressed interest in Drug_B. Average Daily Patients: 48
AGT844,Agent_844,HCP003,Dr. Williams,HCP Details: Name: Dr. Williams Specialty: Oncology Preferred Drug: Drug_C Communication Preference: Phone Preferred Meeting Type: In-Person Meeting Outcome: Positive Product Presented: Drug_A Interaction Type: Virtual Therapeutic Focus: Oncology Agent Name: Agent_844 Agent Product Focus: Drug_B Distance: 88.0 km Drug A Prescriptions: 33 Drug B Prescriptions: 15 Drug C Prescriptions: 92 Other Drugs Prescriptions: 44 Notes: HCP expressed interest in Drug_A. Average Daily Patients: 39
AGT1419,Agent_1419,HCP004,Dr. Brown,HCP Details: Name: Dr. Brown Specialty: Oncology Preferred Drug: Drug_C Communication Preference: Email Preferred Meeting Type: In-Person Meeting Outcome: Positive Product Presented: Drug_C Interaction Type: In-Person Therapeutic Focus: Oncology Agent Name: Agent_1419 Agent Product Focus: Drug_A Distance: 84.0 km Drug A Prescriptions: 45 Drug B Prescriptions: 20 Drug C Prescriptions: 32 Other Drugs Prescriptions: 4 Notes: Discussed benefits of Drug_C. Average Daily Patients: 48
AGT1164,Agent_1164,HCP005,Dr. Taylor,HCP Details: Name: Dr. Taylor Specialty: Dermatology Preferred Drug: Drug_A Communication Preference: Email Preferred Meeting Type: In-Person Meeting Outcome: Neutral Product Presented: Drug_A Interaction Type: Phone Therapeutic Focus: Dermatology Agent Name: Agent_1164 Agent Product Focus: Drug_C Distance: 217.0 km Drug A Prescriptions: 35 Drug B Prescriptions: 91 Drug C Prescriptions: 32 Other Drugs Prescriptions: 73 Notes: Discussed benefits of Drug_A. Average Daily Patients: 30
AGT1287,Agent_1287,HCP006,Dr. Lee,HCP Details: Name: Dr. Lee Specialty: Oncology Preferred Drug: Drug_A Communication Preference: In-Person Preferred Meeting Type: Virtual Meeting Outcome: Neutral Product Presented: Drug_B Interaction Type: In-Person Therapeutic Focus: Oncology Agent Name: Agent_1287 Agent Product Focus: Drug_A Distance: 169.0 km Drug A Prescriptions: 9 Drug B Prescriptions: 23 Drug C Prescriptions: 74 Other Drugs Prescriptions: 43 Notes: HCP needs more data on Drug_B. Average Daily Patients: 25
AGT1451,Agent_1451,HCP007,Dr. Jones,HCP Details: Name: Dr. Jones Specialty: Dermatology Preferred Drug: Drug_B Communication Preference: Email Preferred Meeting Type: In-Person Meeting Outcome: Negative Product Presented: Drug_C Interaction Type: In-Person Therapeutic Focus: Dermatology Agent Name: Agent_1451 Agent Product Focus: Drug_A Distance: 139.0 km Drug A Prescriptions: 81 Drug B Prescriptions: 61 Drug C Prescriptions: 41 Other Drugs Prescriptions: 41 Notes: Scheduled follow-up meeting regarding Drug_C. Average Daily Patients: 41
AGT878,Agent_878,HCP008,Dr. Clark,HCP Details: Name: Dr. Clark Specialty: Dermatology Preferred Drug: Drug_B Communication Preference: Email Preferred Meeting Type: In-Person Meeting Outcome: Neutral Product Presented: Drug_A Interaction Type: Virtual Therapeutic Focus: Dermatology Agent 

In [0]:
from openai import OpenAI
import pandas as pd
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType

# Initialize Spark session
spark = SparkSession.builder.getOrCreate()

# Get Databricks API token
databricks_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
if databricks_token is None:
    raise ValueError("DATABRICKS_TOKEN environment variable is not set")
os.environ['OPENAI_API_KEY'] = databricks_token

# Create the OpenAI client outside of the UDF
client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'],
    base_url="https://dbc-6c2d4d85-4f3f.cloud.databricks.com/serving-endpoints"
)

# Collect the data from the DataFrame
data = final_df.collect()

# Create a list to hold the responses
responses = []

# Loop through the collected data and make API calls
for row in data:
    agent_id = row['Agent_ID']
    hcp_id = row['HCP_ID']
    hcp_details = row['prompt']
    # Combine system prompt with specific HCP details
    combined_prompt = system_prompt + "\n" + hcp_details
    
    response = client.chat.completions.create(
        model="databricks-mixtral-8x7b-instruct",
        # Pass the content as the prompt
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": hcp_details}
        ],
        max_tokens=512,
        temperature=0.7
    )
    
    # Append the original prompt and response to the responses list
    responses.append((agent_id, hcp_id, hcp_details, response.choices[0].message.content.strip()))  

# Convert the responses to a Pandas DataFrame
responses_df = pd.DataFrame(responses, columns=["Agent_ID","HCP_ID","prompt", "response_content"])

# Convert the Pandas DataFrame back to a Spark DataFrame
response_schema = StructType([
    StructField("Agent_ID", StringType(), True),
    StructField("HCP_ID", StringType(), True),
    StructField("prompt", StringType(), True),
    StructField("response_content", StringType(), True)
])

spark_responses_df = spark.createDataFrame(responses_df, schema=response_schema)
final_responses_df = spark_responses_df.select("Agent_ID","HCP_ID","prompt", "response_content")

# Save the DataFrame with responses
spark_responses_df.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("chat_completion_dataset")

# Display the saved table
spark.table("chat_completion_dataset").select("Agent_ID","HCP_ID","prompt", "response_content").display(truncate=False)


Agent_ID,HCP_ID,prompt,response_content
AGT336,HCP001,HCP Details: Name: Dr. Smith Specialty: Oncology Preferred Drug: Drug_A Communication Preference: In-Person Preferred Meeting Type: In-Person Meeting Outcome: Negative Product Presented: Drug_B Interaction Type: Phone Therapeutic Focus: Oncology Agent Name: Agent_336 Agent Product Focus: Drug_A Distance: 135.0 km Drug A Prescriptions: 10 Drug B Prescriptions: 27 Drug C Prescriptions: 80 Other Drugs Prescriptions: 26 Notes: Provided samples of Drug_B. Average Daily Patients: 50,"🌞 Hello Agent_336, ready to impress Dr. Smith? 🤝 ""Sales are contingent upon the attitude of the salesman, not the attitude of the prospect."" – W. Clement Stone 👤 **HCP Profile** - Name: Dr. Smith - Specialty: Oncology - Distance from you: 135.0 km 🚗 - Prefers: In-Person meetings 🤝, Communication: In-Person or Phone 📞 💊 **Prescribing Behavior** - Dr. Smith has been prescribing the following recently: | Drug | Prescriptions | |-------------|-----------------| | Drug A | 10 | | Drug B | 27 | | Drug C | 80 | | Other Drugs | 26 | 📅 **Interaction History** - Last Meeting Outcome: Negative 🙁 - Products Presented: Drug_B 💊 - Recent Notes: Provided samples of Drug_B 🎁 - Interaction Type: Phone 📞 🌟 **Product Preferences** - Preferred Drug: Drug_A 💖 - Therapeutic Focus: Oncology 💉 🤝 **Recommendations for Upcoming Interaction** - Focus on: Drug_A 🎯 - Keep it light and engaging with Dr. Smith 😂, prefer In-Person meetings 🤝, and communicate via In-Person or Phone 📞. 🏆 You’ve got this! All the best! 💪"
AGT843,HCP002,HCP Details: Name: Dr. Johnson Specialty: Oncology Preferred Drug: Drug_C Communication Preference: Email Preferred Meeting Type: In-Person Meeting Outcome: Neutral Product Presented: Drug_B Interaction Type: Virtual Therapeutic Focus: Oncology Agent Name: Agent_843 Agent Product Focus: Drug_C Distance: 162.0 km Drug A Prescriptions: 14 Drug B Prescriptions: 15 Drug C Prescriptions: 31 Other Drugs Prescriptions: 23 Notes: HCP expressed interest in Drug_B. Average Daily Patients: 48,"👋 Hello Agent_843, ready to impress Dr. Johnson? 🩺 ""Sales are contingent upon the attitude of the salesman, not the attitude of the prospect."" – W. Clement Stone 👤 **HCP Profile** - Name: Dr. Johnson 🩺 - Specialty: Oncology 💊 - Distance from you: 162.0 km 🚗 - Prefers: In-Person meetings 🤝, Email communication 📧 💊 **Prescribing Behavior** - Dr. Johnson has been prescribing the following recently: | Drug | Prescriptions | |-------------|-----------------| | Drug A | 14 | | Drug B | 15 | | Drug C | 🏆 **31** 🏆 | | Other Drugs | 23 | 📅 **Interaction History** - Last Meeting Outcome: Neutral 😐 - Products Presented: Drug_B 💊 - Recent Notes: HCP expressed interest in Drug_B 💡 🌟 **Product Preferences** - Preferred Drug: Drug_C 💉 - Therapeutic Focus: Oncology 💊 🤝 **Recommendations for Upcoming Interaction** - Focus on: Drug_C 🎯 - Keep it light and engaging with Dr. Johnson 😄, prefer In-Person meetings 🤝, and communicate via Email 📧. You’ve got this! All the best! 🚀"
AGT844,HCP003,HCP Details: Name: Dr. Williams Specialty: Oncology Preferred Drug: Drug_C Communication Preference: Phone Preferred Meeting Type: In-Person Meeting Outcome: Positive Product Presented: Drug_A Interaction Type: Virtual Therapeutic Focus: Oncology Agent Name: Agent_844 Agent Product Focus: Drug_B Distance: 88.0 km Drug A Prescriptions: 33 Drug B Prescriptions: 15 Drug C Prescriptions: 92 Other Drugs Prescriptions: 44 Notes: HCP expressed interest in Drug_A. Average Daily Patients: 39,"🌮 Hello Agent_844, ready to impress Dr. Williams? 👩‍⚕️ I heard she's a fan of tacos, so maybe bring some for your in-person meeting! 🌮 ""Sales are contingent upon the attitude of the salesman, not the attitude of the prospect."" – W. Clement Stone 👤 **HCP Profile** - Name: Dr. Williams 👩‍⚕️ - Specialty: Oncology 💊 - Distance from you: 88.0 km - Prefers: In-Person meetings 🤝, Communication: Phone 📱 💊 **Prescribing Behavior** - Dr. Williams has been prescribing the following re